In [1]:
# Use naive exact matching algorithm to match artificial reads to genome

# Download the reads

!python -m wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa


Saved under phix.fa


In [45]:
# Read genome function from practical 3

def read_genome(genome_file):
    genome = ''
    with open(genome_file,'r') as genome_file_handle:
        for genome_line in genome_file_handle:
            if not genome_line[0] == '>':
                genome += genome_line.rstrip()
    
    return genome

genome = read_genome("phix.fa")

In [3]:
# 'p' is the fragment of string (pattern) we want to match against the 't' (text) of reference genome
def naive_exact(p, t):
    occurrences = [] # keep track of indices where p matches t
    
    for i in range(len(t) - len(p) + 1): # every position in t where p could start without running past the end of t 
        match = True
        
        for j in range(len(p)):
            if not t[i+j] == p[j]:
                match = False
                break
        
        if match:
            occurrences.append(i)
    
    return occurrences

# test

p = 'TCA'
t = 'ATTCAATCAG'

ne_matches = naive_exact(p, t)
print(ne_matches)
    

[2, 6]


In [5]:
# Generate random reads by taking fragments of genome from random indices

import random

def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    for _ in range(numReads):
        start = random.randint(0, len(genome)-readLen) - 1
        reads.append(genome[start : start+readLen])
    return reads

In [7]:
reads = generateReads(genome, 100, 100)

numMatched = 0 

for read_p in reads:
    list_matches = naive_exact(read_p, genome)
    
    if len(list_matches) > 0:
        numMatched += 1
        
print('%d / %d reads matched' % (numMatched, len(reads)))
        

100 / 100 reads matched


In [8]:
# Beginning of practical 7, matching real reads
# First download the set of real reads

!python -m wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq


Saved under ERR266411_1.first1000.fastq


In [42]:
# Read the fastQ file
def read_fastQ(file_name):
    seq_list  = []
    qual_list = []
    
    with open(file_name,'r') as fastq_handle:
        while True:
            fastq_handle.readline() # the first line, we don't really care what it is
            sequence = fastq_handle.readline().rstrip()
            fastq_handle.readline() # again, a placeholder that we're not concerned about
            quality = fastq_handle.readline().rstrip()
            
            if len(sequence) == 0:
                break
                
            seq_list.append(sequence)
            qual_list.append(quality)
    
    return seq_list, qual_list

real_reads, _ = read_fastQ("ERR266411_1.first1000.fastq")
print(seqs[:5])

['TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC', 'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATACGAAAGTGTTAACTTCTGCGTCATGGACACGAAAAAACTCCC', 'TAAACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTC', 'AACAAGCAGTAGTAATTCCTGCTTTATCAAGATAATTTTTCGACTCATCAGAAATATCCGAAAGTGTTAACTTCTGCGTCATGGAAGCGATAAAACTCTG', 'AGCCGACGTTTTGGCGGCGCAACCTGTGACGACAAATCTGCTCAAATTTATGCGCGCTTCGATAAAAATGATTGGCGTATCCAACCTGCAGAGTTTTATC']


In [44]:
# Genome is double stranded, and the reads could come from either strand
# The way we set our code, we are looking at only one of those strands.

# Match the reverse complement reads to the genome

def get_reverse_complement(s1):
    compl_str = ''
    complement = {'A':'T', 'C':'G', 'G':'C', 'T':'A', 'N':'N'}
    for i in range(len(s1)):
        nuc_at_i = s1[i]
        comp_at_i = complement[nuc_at_i]
        compl_str = comp_at_i + compl_str
    
    return compl_str


for real_read_p in real_reads:
    list_matches = naive_exact(real_read_p, genome)
    #list_matches = naive_exact(real_read_p[:30], genome) # whole reads to reverse complement
    #rev_compl = get_reverse_complement(real_read_p[:30])
    #list_matches.extend(naive_exact(rev_compl, genome))
    
    if len(list_matches) > 0:
        numMatched += 1
        
print('%d / %d real reads (including the reverse complements) matched to our genome' % (numMatched, len(real_reads)))

35 / 1000 real reads (including the reverse complements) matched to our genome


In [38]:
numMatched = 0 

#for real_read_p in real_reads:
#    list_matches = naive_exact(real_read_p, genome)
    
#    if len(list_matches) > 0:
#        numMatched += 1
        
#print('%d / %d real reads matched to our genome' % (numMatched, len(real_reads)))

7 / 1000 real reads matched to our genome


In [15]:
# let's find out why there are so few reads matched the genome
# Is it the sequencing error?

# Take the part of the read (i.e., first 30 bases) and try to match that

#for real_read_p in real_reads:
#    list_matches = naive_exact(real_read_p[:30], genome) # first 30 bases
    
#    if len(list_matches) > 0:
#        numMatched += 1
        
#print('%d / %d real partial reads matched to our genome' % (numMatched, len(real_reads)))


466 / 1000 real reads matched to our genome


1381 / 1000 real partial reads matched to reverse complement of our genome
